In [2]:
import requests


In [3]:
import getpass
username = input('Email:')
password = getpass.getpass('Password:')


Email:kfronczak@gmail.com
Password:········


In [4]:
'''Gets Authentication Token'''
url     = 'https://prod.immedia-semi.com/login'
data    = '{ "password" : "'+password+'", "client_specifier" : "iPhone 9.2 | 2.2 | 222", "email" : "'+username+'" }'
headers = {'Host': 'prod.immedia-semi.com',
           'Content-Type': 'application/json',
          }

response = requests.post(url, headers=headers, data=data)

authtoken = response.json()['authtoken']['authtoken']

In [21]:
'''Gets Network ID and Account ID'''
url     = 'https://prod.immedia-semi.com/networks'
headers = {'Host': 'prod.immedia-semi.com',
           'TOKEN_AUTH': authtoken,
          }

response = requests.get(url, headers=headers)

network_id = response.json()['networks'][0]['id']
account_id = response.json()['networks'][0]['account_id']


In [27]:
'''Checks to see if sync module is online'''
url     = 'https://prod.immedia-semi.com/network/'+str(network_id)+'/syncmodules'
headers = {'Host': 'prod.immedia-semi.com',
           'TOKEN_AUTH': authtoken,
          }

response = requests.get(url, headers=headers)

status   = response.json()['syncmodule']['status']

if status == 'online':
    print('Sync Module Online!')
else:
    print('ERROR: Sync Module is '+status)



Sync Module Online!


In [30]:
'''Arm or Disarm Module'''
status  = 'disarm'
url     =  'https://prod.immedia-semi.com/network/'+str(network_id)+'/'+status
headers = {'Host': 'prod.immedia-semi.com',
           'TOKEN_AUTH': authtoken,
          }
response = requests.post(url, headers=headers)


{'created_at': '2017-01-20T02:08:37+00:00', 'stage_cs_db': None, 'sm_ack': None, 'sync_module_id': 1487, 'stage_lv': None, 'player_transaction': 'rwCN9YrfcQ58Rvx2', 'stage_dev': None, 'sequence': None, 'server': None, 'parent_command_id': None, 'network_id': 2535, 'id': 67829116, 'duration': None, 'transaction': 'LjwgfvsRgpS4gvie', 'command': 'disarm', 'attempts': 0, 'stage_sm': None, 'firmware_id': None, 'execute_time': '2017-01-20T02:08:37+00:00', 'state_condition': 'new', 'updated_at': '2017-01-20T02:08:37+00:00', 'stage_rest': '2017-01-20T02:08:37+00:00', 'diagnostic': False, 'siren_id': None, 'stage_cs_sent': None, 'stage_is': None, 'account_id': 2463, 'stage_vs': None, 'debug': '', 'state_stage': 'rest', 'camera_id': None, 'by_whom': '', 'lfr_ack': None}


In [51]:
'''Get Summary'''
url     = 'https://prod.immedia-semi.com/homescreen'
headers = {'Host': 'prod.immedia-semi.com',
           'TOKEN_AUTH': authtoken,
          }

response = requests.get(url, headers=headers)

jData  = response.json()['devices']

cameras = {}

for i in range(0, len(jData)-1):
    if jData[i]['device_type'] == 'camera':
        cameras[jData[i]['name']] = {'id':jData[i]['device_id'],
                                     'status':jData[i]['active'],
                                     'temperature':jData[i]['temp'],
                                     'battery':jData[i]['battery']}
    
print(cameras)

{'Living Room': {'status': 'disarmed', 'id': 3875, 'temperature': 69, 'battery': 3}}


In [64]:
'''Get list of events and save most recent image'''
url     = 'https://prod.immedia-semi.com/events/network/'+str(network_id)
headers = {'Host': 'prod.immedia-semi.com',
           'TOKEN_AUTH': authtoken,
          }

response = requests.get(url, headers=headers)

jData = response.json()['event']

for i in range(0, len(jData)):
    if 'video_url' in jData[i]:
        last_event = jData[i]['video_url']
        break

if last_event:
    video_url = 'https://prod.immedia-semi.com/'+last_event
    image_url = 'https://prod.immedia-semi.com/'+last_event[:-3]+'jpg'
    
    with open('thumb.jpg', 'wb') as handle:
        response = requests.get(image_url, headers=headers)
        
        if not response.ok:
            print('UH-OH!!')
        
        for block in response.iter_content(1024):
            handle.write(block)

In [65]:
import homeassistant.components.http.const

ImportError: No module named 'homeassistant'